In [1]:
import click
import keras
import math
import numpy
import os
import shutil
import tarfile
import tensorflow
import tqdm
import lzma

Using TensorFlow backend.


In [2]:
numpy.set_printoptions(threshold=numpy.nan)

In [3]:
config = tensorflow.ConfigProto()
config.gpu_options.allow_growth = True
session = tensorflow.Session(config=config)
keras.backend.set_session(session)

In [14]:
def compress_tar(source_dir, output_filename):
    with tarfile.open(output_filename, 'w:gz') as tar:
        tar.add(source_dir, arcname=os.path.basename(source_dir))
    shutil.rmtree(source_dir)

In [15]:
def extract_tar(file_path):
    pass

In [16]:
def read_bits(file_path):
    bits = []
    with open(file_path, 'rb') as f:
        bs = (ord(chr(b)) for b in f.read())
        for b in bs:
            for i in reversed(range(8)):
                bits.append((b >> i) & 1)
    return numpy.array(bits)

In [17]:
def write_bits(file_path, bits):
    pass

In [18]:
def save_models(file_path, models):
    temp_dir = file_path + '#'
    os.makedirs(temp_dir, exist_ok=True)
    meta_file = os.path.join(temp_dir, "meta")
    with open(meta_file, 'w') as meta:
        meta.write("block_size={}\n".format(block_size))
        meta.write("pad_size={}\n".format(pad_size))
    for i, model in enumerate(models):
        temp_file = os.path.join(temp_dir, "block{}".format(i))
        model.save_weights(temp_file)
    compress_tar(temp_dir, file_path)

In [19]:
def load_models(file_path):
    pass

In [20]:
def save_key(file_path, key):
    numpy.save(file_path, key)
    os.rename(file_path + '.npy', file_path)

In [21]:
#def next_suitable_square(n):
#    return int((math.ceil(n ** (1 / 2) / block_size) * block_size) ** 2)

In [22]:
#def next_suitable_cube(n):
#    return int((math.ceil(n ** (1 / 3) / block_size) * block_size) ** 3)

In [23]:
def next_size_1d(n):
    return int(math.ceil(n / block_size) * block_size)

In [24]:
def next_size_2d(n):
    return int(math.ceil(n / block_size ** 2) * block_size ** 2)

In [25]:
def next_size_3d(n):
    return int(math.ceil(n / block_size ** 3) * block_size ** 3)

In [26]:
def init_model_3d(input_shape):
    input_layer = keras.layers.Input(input_shape)
    
    x = keras.layers.Conv3D(8, (input_shape[0] // 2 + 1, input_shape[1] // 2 + 1, input_shape[2] // 2 + 1))(input_layer)
    x = keras.layers.Flatten()(x)
    
    output_layer = keras.layers.Activation('sigmoid')(x)
    
    model = keras.models.Model(input_layer, output_layer)
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model

In [27]:
def random_pad_3d(bits):
    global pad_size
    pad_size = next_size_3d(len(bits)) - len(bits)
    pad_array = numpy.random.randint(2, size=(pad_size,))
    return numpy.concatenate([bits, pad_array])[numpy.newaxis, :]

In [28]:
def num_blocks(data):
    #return int(numpy.round(data.shape[1] ** (1 / 3) / block_size)) ** 3
    return data.shape[1] // block_size ** 3

In [29]:
#def cubify(bits):
#    return bits.reshape(blocks, 1, block_size, block_size, block_size, 1)

In [44]:
def generate_key_cube(data):
    a = int(numpy.round(len(data[0, :])**(1/3)))
    return numpy.random.randint(2, size=(1, a, a, a, 1))

In [31]:
def split_data(data):
    return data.reshape(blocks, 1, data.shape[1] // blocks)

In [32]:
#def training_data_3d(key, data):
#    size = int(numpy.round(len(data) ** (1 / 3)))
#    x = key.reshape(1, size, size, size, 1)
#    y = data.reshape(1, size, size, size, 1)
#    return x, y

In [33]:
def fit_models(x, y):
    models = []
    for i in tqdm.tqdm(range(blocks)):
        model = init_model_3d(x.shape[1:])
        acc = 0
        while acc < 1:
            model.fit(x, y[i, :, :], batch_size=1, epochs=1, verbose=0)
            acc = model.evaluate(x, y[i, :, :], batch_size=1, verbose=0)[1]
            #print("Accuracy:", acc)
        models.append(model)
    return models

In [34]:
block_size = 32

In [35]:
data_bits = read_bits("/home/santiago/Pictures/")

FileNotFoundError: [Errno 2] No such file or directory: '/home/sanz999/ImageNetResNet.py'

In [ ]:
data_bits.shape

In [ ]:
data = random_pad_3d(data_bits)

In [73]:
data = numpy.random.randint(2, size=(1, 255552))

In [74]:
data.shape

(1, 255552)

In [51]:
blocks = num_blocks(data)

In [52]:
key_cube = generate_key_cube(data)

In [53]:
key_cube.shape

(1, 64, 64, 64, 1)

In [ ]:
dataset = split_data(data)

In [ ]:
dataset.shape

In [72]:
input_layer = keras.layers.Input((64, 64, 64, 1))

x = keras.layers.Conv3D(3, (21, 21, 21))(input_layer)
x = keras.layers.Flatten()(x)

output_layer = keras.layers.Activation('sigmoid')(x)

model = keras.models.Model(input_layer, output_layer)
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_23 (InputLayer)        (None, 64, 64, 64, 1)     0         
_________________________________________________________________
conv3d_22 (Conv3D)           (None, 44, 44, 44, 3)     27786     
_________________________________________________________________
flatten_20 (Flatten)         (None, 255552)            0         
_________________________________________________________________
activation_20 (Activation)   (None, 255552)            0         
Total params: 27,786
Trainable params: 27,786
Non-trainable params: 0
_________________________________________________________________


In [ ]:
#the_model = init_model_3d(key_cubes.shape[2:])

In [ ]:
#the_model.summary()

In [75]:
model.fit(key_cube, data, batch_size=1, epochs=1000)

Epoch 1/1000
1/1 [==============================] - 0s - loss: 0.7222 - acc: 0.4997
Epoch 2/1000
1/1 [==============================] - 0s - loss: 1.6268 - acc: 0.5002
Epoch 3/1000
1/1 [==============================] - 0s - loss: 0.7591 - acc: 0.4999
Epoch 4/1000
1/1 [==============================] - 0s - loss: 1.1961 - acc: 0.4998
Epoch 5/1000
1/1 [==============================] - 0s - loss: 1.3023 - acc: 0.4998
Epoch 6/1000
1/1 [==============================] - 0s - loss: 0.8961 - acc: 0.4999
Epoch 7/1000
1/1 [==============================] - 0s - loss: 0.7358 - acc: 0.5002
Epoch 8/1000
1/1 [==============================] - 0s - loss: 1.0177 - acc: 0.5002
Epoch 9/1000
1/1 [==============================] - 0s - loss: 1.0565 - acc: 0.5002
Epoch 10/1000
1/1 [==============================] - 0s - loss: 0.8392 - acc: 0.5004
Epoch 11/1000
1/1 [==============================] - 0s - loss: 0.7166 - acc: 0.5019
Epoch 12/1000
1/1 [==============================] - 0s - loss: 0.8532 - a

1/1 [==============================] - 0s - loss: 0.6998 - acc: 0.5168
Epoch 98/1000
1/1 [==============================] - 0s - loss: 0.6997 - acc: 0.5171
Epoch 99/1000
1/1 [==============================] - 0s - loss: 0.6996 - acc: 0.5174
Epoch 100/1000
1/1 [==============================] - 0s - loss: 0.6994 - acc: 0.5171
Epoch 101/1000
1/1 [==============================] - 0s - loss: 0.6993 - acc: 0.5174
Epoch 102/1000
1/1 [==============================] - 0s - loss: 0.6992 - acc: 0.5179
Epoch 103/1000
1/1 [==============================] - 0s - loss: 0.6990 - acc: 0.5186
Epoch 104/1000
1/1 [==============================] - 0s - loss: 0.6989 - acc: 0.5185
Epoch 105/1000
1/1 [==============================] - 0s - loss: 0.6988 - acc: 0.5187
Epoch 106/1000
1/1 [==============================] - 0s - loss: 0.6987 - acc: 0.5188
Epoch 107/1000
1/1 [==============================] - 0s - loss: 0.6985 - acc: 0.5191
Epoch 108/1000
1/1 [==============================] - 0s - loss: 0.6984

1/1 [==============================] - 0s - loss: 0.6869 - acc: 0.5472
Epoch 193/1000
1/1 [==============================] - 0s - loss: 0.6868 - acc: 0.5476
Epoch 194/1000
1/1 [==============================] - 0s - loss: 0.6867 - acc: 0.5480
Epoch 195/1000
1/1 [==============================] - 0s - loss: 0.6865 - acc: 0.5485
Epoch 196/1000
1/1 [==============================] - 0s - loss: 0.6864 - acc: 0.5488
Epoch 197/1000
1/1 [==============================] - 0s - loss: 0.6862 - acc: 0.5492
Epoch 198/1000
1/1 [==============================] - 0s - loss: 0.6861 - acc: 0.5495
Epoch 199/1000
1/1 [==============================] - 0s - loss: 0.6860 - acc: 0.5499
Epoch 200/1000
1/1 [==============================] - 0s - loss: 0.6858 - acc: 0.5504
Epoch 201/1000
1/1 [==============================] - 0s - loss: 0.6857 - acc: 0.5507
Epoch 202/1000
1/1 [==============================] - 0s - loss: 0.6855 - acc: 0.5510
Epoch 203/1000
1/1 [==============================] - 0s - loss: 0.68

1/1 [==============================] - 0s - loss: 0.6742 - acc: 0.5804
Epoch 288/1000
1/1 [==============================] - 0s - loss: 0.6741 - acc: 0.5806
Epoch 289/1000
1/1 [==============================] - 0s - loss: 0.6740 - acc: 0.5809
Epoch 290/1000
1/1 [==============================] - 0s - loss: 0.6738 - acc: 0.5811
Epoch 291/1000
1/1 [==============================] - 0s - loss: 0.6737 - acc: 0.5814
Epoch 292/1000
1/1 [==============================] - 0s - loss: 0.6736 - acc: 0.5817
Epoch 293/1000
1/1 [==============================] - 0s - loss: 0.6735 - acc: 0.5820
Epoch 294/1000
1/1 [==============================] - 0s - loss: 0.6733 - acc: 0.5823
Epoch 295/1000
1/1 [==============================] - 0s - loss: 0.6732 - acc: 0.5826
Epoch 296/1000
1/1 [==============================] - 0s - loss: 0.6731 - acc: 0.5830
Epoch 297/1000
1/1 [==============================] - 0s - loss: 0.6730 - acc: 0.5835
Epoch 298/1000
1/1 [==============================] - 0s - loss: 0.67

1/1 [==============================] - 0s - loss: 0.6634 - acc: 0.6050
Epoch 383/1000
1/1 [==============================] - 0s - loss: 0.6633 - acc: 0.6052
Epoch 384/1000
1/1 [==============================] - 0s - loss: 0.6632 - acc: 0.6054
Epoch 385/1000
1/1 [==============================] - 0s - loss: 0.6631 - acc: 0.6055
Epoch 386/1000
1/1 [==============================] - 0s - loss: 0.6630 - acc: 0.6056
Epoch 387/1000
1/1 [==============================] - 0s - loss: 0.6629 - acc: 0.6059
Epoch 388/1000
1/1 [==============================] - 0s - loss: 0.6628 - acc: 0.6061
Epoch 389/1000
1/1 [==============================] - 0s - loss: 0.6627 - acc: 0.6062
Epoch 390/1000
1/1 [==============================] - 0s - loss: 0.6626 - acc: 0.6065
Epoch 391/1000
1/1 [==============================] - 0s - loss: 0.6625 - acc: 0.6066
Epoch 392/1000
1/1 [==============================] - 0s - loss: 0.6624 - acc: 0.6069
Epoch 393/1000
1/1 [==============================] - 0s - loss: 0.66

...

1/1 [==============================] - 0s - loss: 0.6488 - acc: 0.6271
Epoch 573/1000
1/1 [==============================] - 0s - loss: 0.6487 - acc: 0.6272
Epoch 574/1000
1/1 [==============================] - 0s - loss: 0.6487 - acc: 0.6272
Epoch 575/1000
1/1 [==============================] - 0s - loss: 0.6486 - acc: 0.6273
Epoch 576/1000
1/1 [==============================] - 0s - loss: 0.6486 - acc: 0.6273
Epoch 577/1000
1/1 [==============================] - 0s - loss: 0.6485 - acc: 0.6274
Epoch 578/1000
1/1 [==============================] - 0s - loss: 0.6485 - acc: 0.6275
Epoch 579/1000
1/1 [==============================] - 0s - loss: 0.6484 - acc: 0.6276
Epoch 580/1000
1/1 [==============================] - 0s - loss: 0.6484 - acc: 0.6277
Epoch 581/1000
1/1 [==============================] - 0s - loss: 0.6483 - acc: 0.6277
Epoch 582/1000
1/1 [==============================] - 0s - loss: 0.6482 - acc: 0.6278
Epoch 583/1000
1/1 [==============================] - 0s - loss: 0.64

1/1 [==============================] - 0s - loss: 0.6444 - acc: 0.6314
Epoch 668/1000
1/1 [==============================] - 0s - loss: 0.6444 - acc: 0.6314
Epoch 669/1000
1/1 [==============================] - 0s - loss: 0.6443 - acc: 0.6315
Epoch 670/1000
1/1 [==============================] - 0s - loss: 0.6443 - acc: 0.6314
Epoch 671/1000
1/1 [==============================] - 0s - loss: 0.6442 - acc: 0.6314
Epoch 672/1000
1/1 [==============================] - 0s - loss: 0.6442 - acc: 0.6315
Epoch 673/1000
1/1 [==============================] - 0s - loss: 0.6442 - acc: 0.6315
Epoch 674/1000
1/1 [==============================] - 0s - loss: 0.6441 - acc: 0.6316
Epoch 675/1000
1/1 [==============================] - 0s - loss: 0.6441 - acc: 0.6316
Epoch 676/1000
1/1 [==============================] - 0s - loss: 0.6441 - acc: 0.6316
Epoch 677/1000
1/1 [==============================] - 0s - loss: 0.6440 - acc: 0.6316
Epoch 678/1000
1/1 [==============================] - 0s - loss: 0.64

1/1 [==============================] - 0s - loss: 0.6414 - acc: 0.6333
Epoch 763/1000
1/1 [==============================] - 0s - loss: 0.6413 - acc: 0.6334
Epoch 764/1000
1/1 [==============================] - 0s - loss: 0.6413 - acc: 0.6334
Epoch 765/1000
1/1 [==============================] - 0s - loss: 0.6413 - acc: 0.6334
Epoch 766/1000
1/1 [==============================] - 0s - loss: 0.6413 - acc: 0.6334
Epoch 767/1000
1/1 [==============================] - 0s - loss: 0.6412 - acc: 0.6335
Epoch 768/1000
1/1 [==============================] - 0s - loss: 0.6412 - acc: 0.6335
Epoch 769/1000
1/1 [==============================] - 0s - loss: 0.6412 - acc: 0.6335
Epoch 770/1000
1/1 [==============================] - 0s - loss: 0.6412 - acc: 0.6335
Epoch 771/1000
1/1 [==============================] - 0s - loss: 0.6411 - acc: 0.6335
Epoch 772/1000
1/1 [==============================] - 0s - loss: 0.6411 - acc: 0.6335
Epoch 773/1000
1/1 [==============================] - 0s - loss: 0.64

1/1 [==============================] - 0s - loss: 0.6393 - acc: 0.6348
Epoch 858/1000
1/1 [==============================] - 0s - loss: 0.6393 - acc: 0.6348
Epoch 859/1000
1/1 [==============================] - 0s - loss: 0.6393 - acc: 0.6348
Epoch 860/1000
1/1 [==============================] - 0s - loss: 0.6392 - acc: 0.6348
Epoch 861/1000
1/1 [==============================] - 0s - loss: 0.6392 - acc: 0.6348
Epoch 862/1000
1/1 [==============================] - 0s - loss: 0.6392 - acc: 0.6348
Epoch 863/1000
1/1 [==============================] - 0s - loss: 0.6392 - acc: 0.6349
Epoch 864/1000
1/1 [==============================] - 0s - loss: 0.6392 - acc: 0.6349
Epoch 865/1000
1/1 [==============================] - 0s - loss: 0.6392 - acc: 0.6349
Epoch 866/1000
1/1 [==============================] - 0s - loss: 0.6391 - acc: 0.6348
Epoch 867/1000
1/1 [==============================] - 0s - loss: 0.6391 - acc: 0.6349
Epoch 868/1000
1/1 [==============================] - 0s - loss: 0.63

1/1 [==============================] - 0s - loss: 0.6379 - acc: 0.6353
Epoch 953/1000
1/1 [==============================] - 0s - loss: 0.6379 - acc: 0.6352
Epoch 954/1000
1/1 [==============================] - 0s - loss: 0.6379 - acc: 0.6352
Epoch 955/1000
1/1 [==============================] - 0s - loss: 0.6379 - acc: 0.6352
Epoch 956/1000
1/1 [==============================] - 0s - loss: 0.6379 - acc: 0.6352
Epoch 957/1000
1/1 [==============================] - 0s - loss: 0.6378 - acc: 0.6352
Epoch 958/1000
1/1 [==============================] - 0s - loss: 0.6378 - acc: 0.6352
Epoch 959/1000
1/1 [==============================] - 0s - loss: 0.6378 - acc: 0.6352
Epoch 960/1000
1/1 [==============================] - 0s - loss: 0.6378 - acc: 0.6352
Epoch 961/1000
1/1 [==============================] - 0s - loss: 0.6378 - acc: 0.6352
Epoch 962/1000
1/1 [==============================] - 0s - loss: 0.6378 - acc: 0.6352
Epoch 963/1000
1/1 [==============================] - 0s - loss: 0.63

In [ ]:
models = fit_models(key_cube, dataset)

In [ ]:
#result = numpy.round(the_model.predict(key_cube)).astype(numpy.uint8)

In [ ]:
#numpy.testing.assert_equal(result, data)

In [ ]:
save_models("/home/sanz999/convcrypt/test.convcrypt", models)

In [ ]:
save_key("/home/sanz999/convcrypt/test.convkey", key_cube)